In [11]:
# numpy and pandas for data manipulation
import pandas as pd 
pd.options.mode.chained_assignment = None # no warnings
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


# import tensorflow, , keras
import sys
sys.path.append('./')
from text_processing import text_processing
from classifiers import batch_classify
import tensorflow_hub as hub

Load Data
Load data using read_csv()

In [3]:
# reading in the CSV files using the pandas read_csv function. we have also dropped the id column from the train set as we won’t need this for training the model.
train = pd.read_csv('./Data/train.csv').drop('id', axis=1)
test = pd.read_csv('./Data/test.csv')
ID = test['id']
test = test.drop('id', axis=1)
test['target']=''


Data Preprocessing
Using the following text preprocessing methods:
[1] Format text: correct spelling, remove emojis, and remove stop_words
[2] Format keyword: fill missing keywords for certain tweets following specific scenarios
[3] Format location: fill missing locations for certain tweets following specific scenarios

** For deatils per each scenario, please follow the text_processing documentation

Scikit-pipeline
Next, we will create a scikit-pipeline with following methods:
[1] CountVectorizer() - split data into tokens, or individual words, count the frequency in which each word appears in the text, and then represent theses counts as a sparse matrix
[2] TfidfTransformer() - apply a weightning to the word counts produced by CountVectorizer(), this weightning is to scale down impact of very frequently occuring words in the text
[3] SGDClassifier() - generally regarded as one of the best text classification algorithms

In [3]:
train_clean = text_processing(train)
test_clean = text_processing(test)
train_clean.to_csv('./Data/train_clean.csv', index=False)
test_clean.to_csv('./Data/test_clean.csv', index=False)


C:\Users\lenovo\anaconda3\envs\ds\lib\site-packages\geopy\point.py:455: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(latitude, longitude, altitude)
C:\Users\lenovo\anaconda3\envs\ds\lib\site-packages\geopy\point.py:455: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(latitude, longitude, altitude)
C:\Users\lenovo\anaconda3\envs\ds\lib\site-packages\geopy\point.py:455: UserWarning: Latitude normalization has been prohibi

In [4]:
train_clean = pd.read_csv('./Data/train_clean.csv')
test_clean = pd.read_csv('./Data/test_clean.csv')

In [5]:
# searching for duplicated tweets with different target and removing them
df_mislabeled = train_clean.groupby(['text']).nunique().sort_values(by='target', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['target'] > 1]['target']
train_clean = train_clean[~train_clean['text'].isin(df_mislabeled.index.tolist())]

In [6]:
y = train_clean['target']
X = train_clean.drop('target', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [56]:

# split the train data into a training and test set
rwad = train_clean['keyword'] + ' ' +train_clean['location']+ ' ' +train_clean['text']
X_train, X_test, y_train, y_test = train_test_split(rwad, train_clean['target'], test_size=13/len(train_clean), random_state=0)

In [60]:
y_train = y_train.to_frame()

try clvkkkkkkkkkkkkkkkkkkkkkk

In [7]:
tweets_pipeline = Pipeline([('CVec', CountVectorizer(stop_words='english')),
                     ('Tfidf', TfidfTransformer())])

X_train_tranformed = tweets_pipeline.fit_transform(X_train)
X_test_tranformed = tweets_pipeline.transform(X_test)

In [13]:

df_results = batch_classify(X_train_tranformed, y_train,X_test_tranformed, y_test)
print(df_results.sort_values(by='Area Under Curve', ascending=False))

ValueError: Found input variables with inconsistent numbers of samples: [3, 5858]

In [ ]:
# ML pipeline
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SGDClassifier())
])

model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(classification_report(y_test, y_predict))


In [38]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


In [41]:

model1 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
model1.fit(X_train, y_train)
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])
y_predict = model1.predict(X_test)
print(classification_report(y_test, y_predict))

ValueError: could not convert string to float: 'desolate'

In [47]:
X_train = X_train(['text'], axis =1)
pipeline_LR = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', LogisticRegression())
])
model = pipeline_LR.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(classification_report(y_test, y_predict))



TypeError: 'Series' object is not callable

In [104]:
pipeline_RFC = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', RandomForestClassifier())
])
model = pipeline_RFC.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86         7
           1       0.83      0.83      0.83         6

    accuracy                           0.85        13
   macro avg       0.85      0.85      0.85        13
weighted avg       0.85      0.85      0.85        13



In [105]:
pipeline_SVC = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SVC())
])
model = pipeline_SVC.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93         7
           1       1.00      0.83      0.91         6

    accuracy                           0.92        13
   macro avg       0.94      0.92      0.92        13
weighted avg       0.93      0.92      0.92        13



In [106]:
pipeline_KNN = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', KNeighborsClassifier(n_neighbors=6))
])
model = pipeline_KNN.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88         7
           1       1.00      0.67      0.80         6

    accuracy                           0.85        13
   macro avg       0.89      0.83      0.84        13
weighted avg       0.88      0.85      0.84        13



Submission

In [12]:
submission_test_clean = test_clean.copy()
submission_test_clean = submission_test_clean['text']


0                             happened terrible car crash
1             heard earthquake different cities stay safe
2         forest fire spot pond geese fleeing street save
3                   apocalypse lighting spokane wildfires
4                     typhoon soudelor kills china taiwan
                              ...                        
3258       earthquake safety los angeles safety fasteners
3259    storm ri worse hurricane cityampothers hardest...
3260                        green line derailment chicago
3261                 meg issues hazardous weather outlook
3262    cityofcalgary activated municipal emergency pl...
Name: text, Length: 3263, dtype: object

In [13]:

submission_test_clean = test_clean.copy()
submission_test_clean = submission_test_clean['text']
submission_test_pred = model.predict(submission_test_clean)

In [ ]:
test.insert(loc=0, column='id', value=ID)
id_col = test['id']
submission_df_1 = pd.DataFrame({"id": id_col,
                               "target": submission_test_pred})
submission_df_1.head()

In [15]:
submission_df_1.to_csv('submission_1.csv', index=False)

In [16]:
train.to_csv('./Data/after_preprocessing.csv', index=False)